# Introduction to TensorFlow
In this notebook we will go through the basics in TensorFlow, build a simple multilayer perceptron and try out the TensorBoard visualization utility.

## Low-level API


### Basics
Here we will see how to write a simple Hello, World! application in TensorFlow and  learn about the basics.


#### Hello, World!

In [0]:
import tensorflow as tf

After importing the module, we can create a Constant operation, which will contain the given value.

In [0]:
hello = tf.constant('Hello, World!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


By printing it we can notice that it won't output its value. The variable `hello` just refers to an operation that we have to evaluate to get its value. To do so we have to create a Session then run it.

In [0]:
sess = tf.Session()
print(sess.run(hello))

b'Hello, World!'


#### Tensors
As we saw, we can't directly ask for the value of a constants. This is because behind the screens by defining them we are building a **[computational graph](https://www.tensorflow.org/api_docs/python/tf/Graph)**. A computational graph consists of
* **[Operations](https://www.tensorflow.org/api_docs/python/tf/Operation)**, which are the nodes of the graph, and
* **[Tensors](https://www.tensorflow.org/api_docs/python/tf/Tensor)**, which are the edges.  These represent the values that will flow through the graph.

A **Tensor** consists of a set of a set of primitive  values shaped into an array. Its **rank** shows the number of dimensions it have, while its **shape** specifies the array's length along each dimension. When evaluated, Tensors are pretty similar to numpy's ndarrays.


Let's see some examples:

In [0]:
import tensorflow as tf
import numpy as np

# rank 0 tensors with shape ()
a = tf.constant(3.14)
b = tf.constant(42, dtype=tf.float32)
c = tf.constant(1)
print(a)
print(b)
print(c)

# rank 1 tensor with shape (3,) and (6)
vec1 = tf.constant([1, 2, 3])
vec2 = tf.constant(np.arange(6), dtype=tf.float32)
print(vec1)
print(vec2)

# rank 2 tensor with shape (3, 1) and (1, 3)
mtx1 = tf.constant([[1.], [3.], [2.]])
mtx2 = tf.constant(np.ones(shape=(1,3), dtype=np.float32))
print(mtx1)
print(mtx2)

# Placeholders are special Tensors.
# Unlike constants, they receive their value in sess.run().
# At this point they are just empty stubs.
x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)
print(x)
print(y)

Tensor("Const_8:0", shape=(), dtype=float32)
Tensor("Const_9:0", shape=(), dtype=float32)
Tensor("Const_10:0", shape=(), dtype=int32)
Tensor("Const_11:0", shape=(3,), dtype=int32)
Tensor("Const_12:0", shape=(6,), dtype=float32)
Tensor("Const_13:0", shape=(3, 1), dtype=float32)
Tensor("Const_14:0", shape=(1, 3), dtype=float32)
Tensor("Placeholder:0", dtype=int32)
Tensor("Placeholder_1:0", dtype=int32)


#### Basic Operations

After defining some tensors, you can do operations with them: 

In [0]:
a_plus_b = a + b
a_plus_2b = tf.add(a, 2*b)
print(a * b)
print(a / b)
mtx1_mtx2 = tf.matmul(mtx1, mtx2)
mtx2_mtx1 = tf.matmul(mtx2, mtx1)
x_plus_y = tf.add(x, y)
x_y = tf.multiply(x, y)

print(x_plus_y)


Tensor("mul_6:0", shape=(), dtype=float32)
Tensor("truediv_2:0", shape=(), dtype=float32)
Tensor("Add_7:0", dtype=int32)


As you can see they are not evaluated yet. Some of them could not even be, even if we wanted! `x_plus_y` is the sum of `x` and `y` but both are just placeholders, they don't have any value yet.

#### Session

To evaluate tensors, instantiate a **[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)** object, informally known as a **session**. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. The `run` function returns the value of the evaluated Tensor.

In [0]:
with tf.Session() as sess:
  print("a + b:", sess.run(a_plus_b))
  print("a + 2*b:", sess.run(a_plus_2b))
  print("mtx1 * mtx2:", sess.run(mtx1_mtx2))
  print("mtx2 * mtx1:", sess.run(mtx2_mtx1))
  
  # When using placeholders, pass their value in the feed_dict parameter
  print("x + y:", sess.run(x_plus_y, feed_dict={x: 3, y: 5}))
  print("x * y:", sess.run(x_y, feed_dict={x: 3, y: 5}))

a + b: 45.14
a + 2*b: 87.14
mtx1 * mtx2: [[1. 1. 1.]
 [3. 3. 3.]
 [2. 2. 2.]]
mtx2 * mtx1: [[6.]]
x + y: 8
x * y: 15


### Building  a Multilayer Perceptron

Here will see how to build a simple neuronal network with TensorFlow using only the low-level API.
![](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)  

It will classify handwritten digits from MNIST. The network will have two dense layers and a final softmax layer with 10 outputs.

In [0]:
# First load the MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [0]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 32
display_step = 50

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input and target tensors
X = tf.placeholder(tf.float32, [None, n_input], name='InputData')
Y = tf.placeholder(tf.float32, [None, n_classes], name='LabelData')

In [0]:
# Store layers weight & bias
weights = {
  'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
  'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
  'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
  'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
  'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Create the model
def neural_net(x):
  # A fully connected layer with 256 neurons.
  # Remember, a fully connected layer's formula is
  #     o = g(x*W+b)
  # where g is the activation function, W is weight matrix and b is the bias vector.
  
  layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
  layer_1 = tf.nn.relu(layer_1)
  
  layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
  layer_2 = tf.nn.relu(layer_2)
  
  out_layer = tf.matmul(layer_2, weights['w3']) + biases['b3']
  return out_layer

In [0]:
# Encapsulating all ops into scopes, making Tensorboard's Graph
# Visualization more convenient (see next section about Tensorboard).
with tf.name_scope('Model'):
  # Build model
  logits = neural_net(X)

# Define loss and optimizer
with tf.name_scope('Loss'):
  # Softmax Cross entropy (cost function)
  # This is the cross entropy loss and softmax activation fused into one operation
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                labels=Y))
# The training operations
with tf.name_scope('Adam'):
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss)
  
# Create operations to calculate the accuracy
with tf.name_scope('Accuracy'):
  correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [0]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            l, acc = sess.run([loss, accuracy], 
                              feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(l) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 810.3694, Training Accuracy= 0.188
Step 50, Minibatch Loss= 36.6873, Training Accuracy= 0.906
Step 100, Minibatch Loss= 76.7652, Training Accuracy= 0.812
Step 150, Minibatch Loss= 32.2930, Training Accuracy= 0.938
Step 200, Minibatch Loss= 34.3492, Training Accuracy= 0.906
Step 250, Minibatch Loss= 30.2908, Training Accuracy= 0.844
Step 300, Minibatch Loss= 23.0249, Training Accuracy= 0.906
Step 350, Minibatch Loss= 30.5035, Training Accuracy= 0.906
Step 400, Minibatch Loss= 35.8833, Training Accuracy= 0.906
Step 450, Minibatch Loss= 20.7671, Training Accuracy= 0.969
Step 500, Minibatch Loss= 15.0355, Training Accuracy= 0.938
Optimization Finished!
Testing Accuracy: 0.8978


## TensorBoard
With TensorBoard you can easily monitor the learning process. Unfortunately in Google Colab you cannot directly use TensorBoard, first you have to tunnel it to become  remotely accessible.

### Setup
By running the following commands you can tunnel TensorBoard to the outside world.

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-07 08:58:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.123.104, 52.203.66.95, 52.22.145.207, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.123.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  14.3MB/s    in 1.0s    

2019-04-07 08:58:32 (14.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


The following will give you an url. By opening it you can access your remote TensorBoard.

Modify *LOG_DIR* if needed.

In [0]:
LOG_DIR = './log_test'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6007 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://2427574d.ngrok.io


### Visualization
You can also visualize the computational graph by using TensorBoard. First you have to save the graph to a summary file:


In [0]:
writer = tf.summary.FileWriter('./log_test')
writer.add_graph(tf.get_default_graph())
writer.flush()

Now open the above link and select Graph in the drop-down menu in the top right corner. It will show your computational graph.

If you are running this notebook on your own machine, you can just use the following command:

In [0]:
!tensorboard --logdir ./log_test

## High-level API
Keras is also available as a high-level API for TensorFlow. You can use it by importing the **[tf.keras](https://www.tensorflow.org/guide/keras)** module, but be aware that it might not be the latest Keras version available from PyPi.

Before you move on runtime reset may be necessary because of the visualization. You can do it at: **Runtime -> Reset all runtimes...**. (Warning: You will lose all data from your virtual machine.) After that do not run any previous block except the [remote setup](#scrollTo=9cdJm9TXGdkL).

#### TensorBoard Callback
Using **[tf.keras.callbacks.TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard)** we can easily log information during training, that can be later visualized using TensorBoard. It allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.

**Main arguments:**

*  **log_dir:** the path of the directory where to save the log files to be parsed by TensorBoard.
* **histogram_freq:** frequency (in epochs) at which to compute activation and weight histograms for the layers of the model. If set to 0, histograms won't be computed. Validation data (or split) must be specified for histogram visualizations.
* **batch_size:** size of batch of inputs to feed to the network for histograms computation.
* **write_graph:** whether to visualize the graph in TensorBoard. The log file can become quite large when write_graph is set to True.
* **write_grads:** whether to visualize gradient histograms in TensorBoard.  histogram_freq must be greater than 0.
* **write_images:** whether to write model weights to visualize as image in TensorBoard.


In [0]:
from tensorflow.keras.callbacks import TensorBoard

batch_size = 32

tbCallBack = TensorBoard(log_dir='./log_keras', 
                         histogram_freq=1,
                         batch_size=batch_size,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

#### Visualization
With TensorBoard you can see details about the network during and after training. 

In [0]:
LOG_DIR = './log_keras'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6007 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://953864d8.ngrok.io


#### Build an MLP
But for now we have nothing to see, so let's build and train a simple neural network. During trainingTensorBoard will automatically update.

In [0]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=batch_size, 
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
model.evaluate(x_test, y_test)

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 17s 277us/sample - loss: 0.2968 - acc: 0.9129 - val_loss: 0.1430 - val_acc: 0.9577
Epoch 2/5
60000/60000 [==============================] - 15s 249us/sample - loss: 0.1423 - acc: 0.9577 - val_loss: 0.1050 - val_acc: 0.9672
Epoch 3/5
60000/60000 [==============================] - 17s 282us/sample - loss: 0.1062 - acc: 0.9678 - val_loss: 0.0848 - val_acc: 0.9744
Epoch 4/5
60000/60000 [==============================] - 15s 255us/sample - loss: 0.0880 - acc: 0.9726 - val_loss: 0.0750 - val_acc: 0.9764
Epoch 5/5
10000/10000 [==============================] - 0s 44us/sample - loss: 0.0766 - acc: 0.9766


[0.07656257087504491, 0.9766]

## References
* Low-level introduction: https://www.tensorflow.org/guide/low_level_intro
* Tutorials: https://www.tensorflow.org/tutorials/
* Examples: https://github.com/aymericdamien/TensorFlow-Examples
* TensorBoard in Google Colab: https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
